# Cравнение инструментов для идентификации сложных слов немецкого языка

*Проект подготовили:*
*  <b>Ященко Анастасия</b> — идея проекта, код, подготовка стандарта
*  <b>Соколова Ирина</b> — код, поиск инструментов
*  <b>Краснов Станислав</b> — код, поиск инструментов, оформление

В рамках научного проекта "Semantic Brain Map" возникла задача грамматической разметки текста на немецком языке: в частности, была выявлена необходимость определения каждой лексемы как сложносоставной (то есть состоящей из 2 или более <i>корней</i>, без учёта приставок и суффиксов). Настоящий проект посвящён сравнению двух самых популярных инструментов для обработки сложных слов — CharSplit и Compound Word Splitter. Результат их работы сравнивается с подготовленным списком всех сложных лемм, входящих в текст.

Мы токенизировали текст и лемматизировали его, используя библиотеку SpaCy (все слова теперь в файле lemmas.txt), затем этот список был предложен носителю языка для того, чтобы в файле checking_lst остались только сложные леммы. 

In [5]:
import re
f1 = open('lemmas.txt')
f2 = open('checking_lst.txt')
f1 = f1.readlines()
f2 = f2.readlines()
l = [] #список всех лемм из файла
checking_lst = [] #проврочный список
for line in f1:
    line = re.sub(r'\n','',line)
    l.append(line)
for line in f2:
    line = re.sub(r'\n','',line)
    checking_lst.append(line)
    
lemmas = set(l)  #тут все уникальные леммы
print(len(lemmas))

2622


In [6]:
def evaluate(all_words, results, correct_answers):
    tp = [w for w in all_words if w in results and w in correct_answers]
    fp = [w for w in all_words if w in results and w not in correct_answers]
    tn = [w for w in all_words if w not in results and w not in correct_answers]
    fn = [w for w in all_words if w not in results and w in correct_answers]
    precision = len(tp) / (len(tp) + len(fp))
    recall = len(tp) / (len(tp) + len(fn))
    accuracy = (len(tp) + len(tn)) / (len(tp) + len(tn) + len(fp) + len(fn))
    f_measure = 2 * precision * recall / (precision + recall)
    print('Precision: %.2f' % precision)
    print('Recall: %.2f' % recall)
    print('Accuracy: %.2f' % accuracy)
    print('F-measure: %.2f' % f_measure)
    return

# Сплиттеры

Сплиттер compound-word-splitter (CWS) — это подмодуль спеллчекера Enchant. CWS разделяет слова, которые не смог распознать Enchant, на все возможные комбинации двух подстрок, возвращая лишь те из них, что есть в словаре Enchant. Кроме того, CWS учитывает тот факт, что некоторые сложносоставные слова в немецком языке соединяются с помощью согласного <i>-s-</i>. Если CWS не находит в слове подстрок из словаря Enchant, то просто возвращает пустую строку. Очевидно, что с незнакомыми (словарю Enchant'а) словами этот сплиттер работает плохо (никак).

CWS сам по себе — довольно удобный и полезный инструмент, который хорошо делит, например, слова с приставками, образованные от тех основ, что есть в словаре. Однако, как показала практика, слова с несколькими основами таким алгоритмом делятся гораздо хуже.

In [ ]:
import splitter
compound_word_splitter = [] #список результатов
for lemma in lemmas:
    x = splitter.split(lemma, 'de_de')
    if x != '': 
        #поскольку сплиттер возвращает пустую строку в случае, если слово не является сложным (по enchant'у),
        #то мы просто добавляем в результирующий список все леммы, которые вызвали НЕ пустую строку
        compound_word_splitter.append(lemma)

Сплиттер char_split находит самые вероятные варианты разбиения сложного слова на составные части. Char_split вычисляет вероятность появления нграммы в начале, в середине и в конце слова и таким образом находит место в слове, которое с наибольшей вероятностью будет стыком двух корней: это место, где низка вероятность середины слова, перед которым идёт нграмма с высокой вероятностью конца слова, а после которого идёт нграмма с высокой вероятностью начала слова. 
Сначала метод извлекает нграммы длиной от 4 до n в начале слова, в середине слова и в конце. Получив нграммы и вероятности их появления в разных позициях, считается условная вероятность позиции при появлении конкретной нграммы. 
Затем окно размера от 4 до n, где n равно длина слова минус 3 (минимальная длина части составного слова), перемещается по слову; для каждой позиции в слове вычисляется вероятность того, что здесь может быть стык. Вероятность вычисляется по формуле:
score(n) = max p(prefix) + max p(suffix) − min p(infix)
 
 Модель обучена на 10 миллионах существительных из газет и достигает accuracy примерно 95%. Надо заметить, что char_split был создан, чтобы выделять в сложных словах вершину в случаях, когда сложное слово целиком не представлено в словаре. Успехом считалось, если сплиттер правильно выделял вершину в слове, а что осталось слева, было неважно. При таких критериях модель работает лучше, чем сплиттеры, основанные на правилах. Однако в тестовом множестве все слова были сложными, поэтому неизвестно, как хорошо этот метод находил бы сложные слова среди разных слов. 

Как мы использовали char_split:
char_split возвращает список вариантов разбиения слова с их вероятностями. У каждого слова разное количество вариантов разбиения, и мы считаем, что чем больше вариантов, тем больше вероятность, что слово сложное. Если вариантов больше пяти, мы добавляем лемму в наш список. *(Сначала пробовали по 4 варианта, затем эмпирическим путём пришли к 5-ти.)*

In [ ]:
import char_split
char_split_splitter = [] #список результатов
for lemma in lemmas:
    x = char_split.split_compound(lemma)
    if len(x) > 5:
        #этот сплиттер возвращает список списков корней с вероятностями. Чем слово ближе к сложносоставному, 
        #тем больше вариантов разбиения на основы у него есть. Опытным путём мы решили, что если есть 
        #больше 5 вариантов разбить слово, то оно, скорее всего, сложное (и мы добавляем лемму в наш список).
        char_split_splitter.append(str(lemma))

Теперь посмотрим на оценку работы сплиттеров (выдача алгоритмов сохранена в отдельных файлах *compound_word_splitter_file.txt*, *char_splitter_file_4.txt* и *char_splitter_file_5.txt*).

In [7]:
with open('compound_word_splitter_file.txt') as f:
    compound = f.read().split('\n')

print(len(compound))

1995


In [8]:
with open('char_splitter_file_4.txt') as f:
    char_4 = f.read().split('\n')
print(len(char_4))

605


In [9]:
with open('char_splitter_file_5.txt') as f:
    char_5 = f.read().split('\n')
print(len(char_5))

411


In [10]:
print("Результаты Compound Word Splitter")
print(evaluate(lemmas, compound, checking_lst))

Результаты Compound Word Splitter
Precision: 0.15
Recall: 0.69
Accuracy: 0.30
F-measure: 0.25
None


In [13]:
print("Результаты CharSplit (больше 4 вариантов)")
print(evaluate(lemmas, char_4, checking_lst))

Результаты CharSplit (больше 4 вариантов)
Precision: 0.66
Recall: 0.89
Accuracy: 0.90
F-measure: 0.76
None


In [14]:
print("Результаты CharSplit (больше 5 вариантов)")
print(evaluate(lemmas, char_5, checking_lst))

Результаты CharSplit (больше 5 вариантов)
Precision: 0.80
Recall: 0.73
Accuracy: 0.92
F-measure: 0.76
None


Как видно из результатов, CharSplit даёт огромную фору CWS, демонстрируя F-меру в три раза больше, чем у своего "конкурента" (0.76 против 0.25). При этом, строго говоря, увеличение в CharSplit кол-ва вариантов для леммы с 4 до 5 не привело к каким-либо значительным качественным изменениям (хотя и уменьшило размер выдачи на треть). Дальнейшее увеличение кол-ва вариантов не приводит к положительным результатам, а лишь уменьшает оценку.